# Practicant amb training i test sets

**Descripció**
Familiaritza't amb la programació científica mitjantçant la llibreria SKLearn / Scikitlearn.

In [21]:
import pandas as pd
import numpy as np
import random

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.decomposition import PCA

## Nivell 1

### Exercici 1
Parteix el conjunt de dadesDelayedFlights.csv en train i test. Estudia els dos conjunts per separat, a nivell descriptiu.

## Nivell 2
### Exercici 2
Aplica algun procés de transformació (estandarditzar les dades numèriques, crear columnes dummies, polinomis...).

## Nivell 3
### Exercici 3
Resumeix les noves columnes generades de manera estadística i gràfica